# How to use the Seshat API

First import the package and set up a client.

In [8]:
from seshat_api import SeshatAPI
import pandas as pd

In [9]:
# client = SeshatAPI(base_url="http://localhost:8000/api")  # Use this if you are running the API locally
client = SeshatAPI(base_url="https://seshatdata.com/api")  # Use this if you are using the public API (seshatdata is a test server)

In [10]:
# Function to get all pages of data from the API for a given function
def get_seshat_data(client, data_func):
    data = data_func(client)
    all_results = []
    page = 1

    while True:
        try:
            response = data.get_page(page)
        except:
            break
        results = response.get('results', [])
        if not results:
            break
        all_results.extend(results)
        page += 1

    return pd.DataFrame(all_results)

As a first test, let's import `Polities` from the `core` app and look at them in pandas:

In [11]:
from seshat_api.core import Polities
polities_df = get_seshat_data(client, Polities)

In [12]:
# polities_df.head()

Now let's try something more interesting! I want to know which polities in the Seshat database are recorded as having the religious tradition of Christianity in the year 1500 and add them to a pandas dataframe.

In [13]:
from seshat_api.general import PolityReligiousTraditions
religious_traditions_df = get_seshat_data(client, PolityReligiousTraditions)
# religious_traditions_df.head()

In [14]:
# Extract the polities column to a new dataframe
polities_with_religious_traditions_df = pd.DataFrame(religious_traditions_df['polity'].tolist())

# Add the religious tradition column to the new dataframe
polities_with_religious_traditions_df['religious_tradition'] = religious_traditions_df['religious_tradition']

# Filter by the religious tradition of Christianity
polities_with_christianity_df = polities_with_religious_traditions_df[polities_with_religious_traditions_df['religious_tradition'] == 'Christianity']

# Filter by polities active in the year 1500
polities_with_christianity_1500_df = polities_with_christianity_df[(polities_with_christianity_df['start_year'] <= 1500) & (polities_with_christianity_df['end_year'] >= 1500)]

In [ ]:
polities_with_christianity_1500_df[['long_name', 'religious_tradition']]